## Initialization

In [ ]:
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import matplotlib.pyplot as plt
from tqdm import tqdm

root_dir = os.getcwd().split("AdversarialNIDS")[0] + "AdversarialNIDS"
sys.path.append(root_dir)

from scripts.logger import LoggerManager
from scripts.analysis.model_analysis import perform_model_analysis

from CICIDS2017.preprocessing.dataset import CICIDS2017
from UNSWNB15.preprocessing.dataset import UNSWNB15

from scripts.models.pytorch.MLP import NetworkIntrusionMLP
from scripts.models.pytorch.CNN import NetworkIntrustionCNN
from scripts.models.pytorch.LSTM import NetworkIntrusionLSTM

from scripts.models.pytorch.train import train
from scripts.models.pytorch.visualization import display_loss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lm = LoggerManager(log_dir=f"{root_dir}/logs", log_name="TDM")
logger = lm.get_logger()
title = lm.get_title()
logger.info(f"Logger initialized for '{title}'")

## Initialization of the Dataset

In [ ]:
dataset = CICIDS2017( # [UNSWNB15() or CICIDS2017()]
    dataset_size="full",
    logger=logger
).optimize_memory().encode(attack_encoder="label").scale(scaler="minmax")

In [ ]:
X_train, X_val, y_train, y_val = dataset.subset(size=50000, multi_class=True).split(
    one_hot=True,
    apply_smote=True,
    to_tensor=True
)

In [ ]:
# Create DataLoaders
train_dataset = TensorDataset(X_train.to(device), y_train.to(device))
val_dataset = TensorDataset(X_val.to(device), y_val.to(device))

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
input_size = train_loader.dataset.tensors[0].shape[1]
num_classes = train_loader.dataset.tensors[1].shape[1]
print(f"Input size: {input_size}, Num classes: {num_classes}")  

criterion = nn.CrossEntropyLoss()

## Multi Layers Perceptron (MLP)

In [ ]:
model_mlp = NetworkIntrusionMLP(input_size=input_size, num_classes=num_classes).to(device)
logger.info(f"MLP Model initialized with {model_mlp.num_params()} parameters")

learning_rate_mlp = 1e-2
num_epochs_mlp = 100

optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=learning_rate_mlp)
scheduler_mlp = optim.lr_scheduler.ReduceLROnPlateau(optimizer_mlp, mode='min', factor=0.8, patience=10, min_lr=1e-5)

In [ ]:
model_mlp, train_losses_mlp, val_losses_mlp = train(
    model=model_mlp,
    optimizer=optimizer_mlp,
    scheduler=scheduler_mlp,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_mlp
)

In [ ]:
display_loss(
    list_epoch_loss=train_losses_mlp,
    list_val_loss=val_losses_mlp,
    title=f"{title}_MLP",
    dir=root_dir,
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_mlp,
    X_test=X_val,
    y_test=y_val,
    logger=logger,
    model_name=f"{title}_MLP",
    dir=root_dir,
    plot=True,
    device=device
)

## Convolutional Neural Network (CNN)

In [ ]:
model_cnn = NetworkIntrustionCNN(input_channels=1, input_size= input_size, num_classes=num_classes).to(device)
logger.info(f"CNN Model initialized with {model_cnn.num_params()} parameters")

learning_rate_cnn = 1e-2
num_epochs_cnn = 100

optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=learning_rate_cnn)
scheduler_cnn = optim.lr_scheduler.ReduceLROnPlateau(optimizer_cnn, mode='min', factor=0.8, patience=10, min_lr=1e-5)

In [ ]:
model_cnn, train_loss_cnn, val_loss_cnn = train(
    model=model_cnn,
    optimizer=optimizer_cnn,
    scheduler=scheduler_cnn,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_cnn
)

In [ ]:
display_loss(
    train_loss_cnn,
    val_loss_cnn,
    title=f"{title}_CNN",
    dir=root_dir,
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_cnn,
    X_test=X_val,
    y_test=y_val,
    model_name=f"{title}_CNN",
    dir=root_dir,
    plot=True,
    logger=logger,
    device=device
)

## Long Short-Term Memory (LSTM)

In [ ]:
model_lstm = NetworkIntrusionLSTM(input_size=input_size, hidden_size=64, num_layers=3, num_classes=num_classes).to(device)
logger.info(f"LSTM Model initialized with {model_lstm.num_params()} parameters")

learning_rate_lstm = 1e-2
num_epochs_lstm = 100

optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=learning_rate_lstm)
scheduler_lstm = optim.lr_scheduler.ReduceLROnPlateau(optimizer_lstm, mode='min', factor=0.8, patience=10, min_lr=1e-5)

In [ ]:
model_lstm, train_loss_lstm, val_loss_lstm = train(
    model=model_lstm,
    optimizer=optimizer_lstm,
    scheduler=scheduler_lstm,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_lstm
)

In [ ]:
display_loss(
    train_loss_lstm, 
    val_loss_lstm, 
    title=f"{title}_LSTM",
    dir=root_dir, 
    logger=logger,
    epoch_min=2
)

In [ ]:
cm, cr = perform_model_analysis(
    model=model_lstm,
    X_test=X_val,
    y_test=y_val,
    model_name=f"{title}_LSTM",
    dir=root_dir,
    plot=True,
    logger=logger,
    device=device
)